In [1]:
import numpy as np
from tensorflow import keras
fashion_mnist = keras.datasets

import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import urllib
urllib.request.urlretrieve ("https://gitlab.com/knork/data/raw/e0a833fb698e00db25be29dba89f0b6c6a2efcc8/ORL_faces.npz?inline=false", "face.npz")

data = np.load('face.npz')
trainX = data['trainX']
trainY = data['trainY']
testX = data['testX']
testY = data['testY']
trainX_check= np.reshape(trainX,(240,112,92))
plt.imshow(trainX_check[0])
plt.show()
import torch

<Figure size 640x480 with 1 Axes>

In [2]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, H)
        self.out = torch.nn.Linear(H, D_out)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.softmax(self.out(x),dim=1)
        return x

3.0244290828704834
Accuracy of the network on the 10000 test images: 17 %


In [ ]:
class MySGD(torch.optim.Optimizer):
  
    def __init__(self, params, lr=1e-4):
        defaults = dict(lr=lr)
        super(MySGD, self).__init__(params, defaults)    
    
    def step(self, closure=None):        
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:   
            
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                p.data.add_(-group['lr'], d_p)

        return loss


In [ ]:
dtype = torch.float
device = torch.device("cpu")
device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, trainX[0].size, 512, 20

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in,dtype=dtype,device=device)
y = torch.randn(N, D_out,dtype=dtype,device=device)
x = torch.from_numpy(trainX).float().to(device)
y = torch.from_numpy(trainY)
y=y.type(torch.LongTensor).to(device)



# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)
model.cuda()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, nesterov=True, momentum=0.3)
optimizer = MySGD(model.parameters())

In [ ]:
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x).squeeze()

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 1000 == 99:
        print(loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

test_x = torch.from_numpy(testX).float().to(device)
test_y = torch.from_numpy(testY).to(device)


correct = 0
total = 0
with torch.no_grad():
    outputs = model(test_x)
    _, predicted = torch.max(outputs.data, 1)
    total += test_y.shape[0]
    correct += (predicted == test_y).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))